# Spring Security OAuth2
doc:
- [for servlet](https://docs.spring.io/spring-security/reference/servlet/oauth2/index.html)
- [OAuth2 WebFlux](https://docs.spring.io/spring-security/reference/reactive/oauth2/index.html)

books:
* Spilca, Laurentiu. **Spring Security in Action**. 2024. Manning.
  * authorization server: 8080
  * resource server: 9090
  * client: 8080, authorization server: 7070

actions: 
* example-springcloud/auth/oauth2-authorization-server
* example-springcloud/auth/oauth2-resource-server
* example-springcloud/auth/oauth2-resource-client
* HTTPie: Archived > Spring Security in Action

example: 
- [Spring Cloud Gateway + OAuth2](https://juejin.cn/post/7137649100115148814)

# Components
- OAuth2 Resource Server
- OAuth2 Client: OAuth2 Log in
- Spring Authorization Server: seperate project built on Spring Security

# Authorization Server
* [Doc](https://docs.spring.io/spring-authorization-server/reference/index.html)
  * How-to Guides
* code: `D:\workspace\rtfsc\spring-authorization-server`

```groovy
// with Spring Boot
implementation "org.springframework.boot:spring-boot-starter-oauth2-authorization-server"

// without Spring Boot
implementation "org.springframework.security:spring-security-oauth2-authorization-server"
```

## Configuration Model

```
OAuth2AuthorizationServerConfiguration
OAuth2AuthorizationServerConfigurer
|--  RegisteredClientRepository                                + manage new and existing clients
|--  OAuth2AuthorizationService                                - manage new and existing authorization
|--  OAuth2AuthorizationConsentService                         - manage new and existing authorization consents
|--  AuthorizationServerSettings                               + customize OAuth2 authorization server configuration settings
|-- |-- issuer
|--  OAuth2TokenGenerator                                      - generate tokens supported by OAuth2 authorization server
|--  OAuth2ClientAuthenticationConfigurer                      - OAuth2 client authentication
|-- |-- AuthenticationConverter                                 - (pre-processor) extract client credentials from HttpServletRequst to OAuth2ClientAuthenticationToken
|-- |-- AuthenticationProvider                                  - (main-processor) authenticate OAuth2ClientAuthenticationToken
|-- |-- |-- JwtClientAssertionAuthenticationProvider
|-- |-- |-- JwtClientAssertionDecoderFactory                        - Function<RegisteredClient, OAuth2TokenValidator<Jwt>>
|-- |-- AuthenticationSuccessHandler                            - (post-processor) successful client authentication to SecurityContext
|-- |-- AuthenticationFailureHandler                            - (post-processor) failed client authentication to return OAuth2Error
|-- |-- OAuth2ClientAuthenticationFilter                        - (Filter) porcess client authentication requests
|-- |-- |-- AuthenticationConverter                                 - DelegatingAuthenticationConverter
|-- |-- |-- |-- JwtClientAssertionAuthenticationConverter
|-- |-- |-- |-- ClientSecretBasicAuthenticationConverter
|-- |-- |-- |-- ClientSecretPostAuthenticationConverter
|-- |-- |-- |-- PublicClientAuthenticationConverter
|-- |-- |-- AuthenticationManager
|-- |-- |-- |-- JwtClientAssertionAuthenticationProvider
|-- |-- |-- |-- ClientSecretAuthenticationProvider
|-- |-- |-- |-- PublicClientAuthenticationProvider
|-- |-- |-- AuthenticationSuccessHandler                            - OAuth2ClientAuthenticationToken
|-- |-- |-- AuthenticationFailureHandler                            - OAuth2AuthenticationException
|--  OAuth2AuthorizationEndpointConfigurer                     - OAuth2 authorization endpoint
|--  OAuth2DeviceAuthorizationEndpointConfigurer               - OAuth2 device authorization endpoint
|--  OAuth2DeviceVerificationEndpointConfigurer                - OAuth2 device verification endpoint
|--  OAuth2TokenEndpointConfigurer                             - OAuth2 token endpoint
|--  OAuth2TokenIntrospectionEndpointConfigurer                - OAuth2 token introspection endpoint
|--  OAuth2TokenRevocationEndpointConfigurer                   - OAuth2 token revocation endpoint
|--  OAuth2AuthorizationServerMetadataEndpointConfigurer       - OAuth2 authorization server metadata endpoint
|--  OidcConfigurer                                            - OpenID Connnect 1.0 
|-- |--  OidcProviderConfigurationEndpointConfigurer             - provider configuration endpoint
|-- |--  OidcLogoutEndpointConfigurer                            - logout endpoint
|-- |--  OidcUserInfoEndpointConfigurer                          - UserInfo endpoint
|-- |--  OidcClientRegistrationEndpointConfigurer                - client registration endpoint
|--  NimbusJwkSetEndpointFilter

AuthorizationServerContext
AuthorizationServerContextHolder
```

## Code Model/Components

```
RegisteredClient
|-- id  
|-- clientId    
|-- clientIdIssuedAt   
|-- clientSecret    
|-- clientSecretExpiresAt  
|-- clientName  
|-- clientAuthenticationMethods     - ClientAuthenticationMethod: client_secret_basic, client_secret_post, client_secret_jwt
|-- authorizationGrantTypes         - AuthorizationGrantType: authorization_code, refresh_token, client_credentials, password, jwt-bearer, device_code
|-- redirectUris   
|-- postLogoutRedirectUris 
|-- scopes 
|-- clientSettings                  - ClientSettings
|-- tokenSettings                   - TokenSettings
RegisteredClientRepository
|-- InMemoryRegisteredClientRepository
|-- JdbcRegisteredClientRepository

OAuth2Authorization
|-- id
|-- registeredClientId
|-- principalName
|-- authorizationGrantType          - AuthorizationGrantType
|-- authorizedScopes
|-- tokens                          - OAuth2Token
|-- |-- OAuth2 authorization_code: OAuth2AuthorizationCode, OAuth2AccessToken, OAuth2RefreshToken
|-- |-- OpenID Connect 1.0 authorization_code: OAuth2AuthorizationCode, OidcIdToken, OAuth2AccessToken, OAuth2RefreshToken
|-- |-- OAuth2 client_credentials: OAuth2AccessToken
|-- attributes
OAuth2AuthorizationService
|-- InMemoryOAuth2AuthorizationService
|-- JdbcOAuth2AuthorizationService

OAuth2AuthorizationConsent          - authorization consent(decision)
|-- registeredClientId
|-- principalName
|-- authorities                     - GrantedAuthority
OAuth2AuthorizationConsentService
|-- InMemoryOAuth2AuthorizationConsentService
|-- JdbcOAuth2AuthorizationConsentService

OAuth2TokenContext                  - hold info associated with OAuth2Token
|-- getRegisteredClient                 - RegisteredClient 
|-- getPrincipal                        - Authentication
|-- getAuthorizationServerContext       - AuthorizationServerContext
|-- getAuthorization                    - OAuth2Authorization
|-- getAuthorizedScopes                 - Set<String>
|-- getTokenType                        - OAuth2TokenType: code, access_token, refresh_token, id_token
|-- getAuthorizationGrantType           - AuthorizationGrantType
|-- getAuthorizationGrant               - Authentication
OAuth2TokenGenerator                - generate OAuth2Token from OAuth2TokenContext
|-- OAuth2TokenFormat                   - self-contained, reference
|-- ClaimAccessor
|-- OAuth2AccessTokenGenerator
|-- OAuth2RefreshTokenGenerator
|-- JwtGenerator
OAuth2TokenCustomizer               - customize the attributes of OAuth2Token
|-- OAuth2TokenClaimsContext
|-- JwtEncodingContext

SessionRegistry                     - track OpenID Connect 1.0 authenticated sessions
|-- SessionRegistryImpl                 - default, with HttpSessionEventPublisher
|-- SessionAuthenticationStrategy       - associated to OAuth2 authorization endpoint
|-- SessionInformation                  - associated to End-User
```

## Protocol Endpoints

```java
public final class AuthorizationServerSettings extends AbstractSettings {
	public static Builder builder() {
		return new Builder()
				.authorizationEndpoint("/oauth2/authorize")
				.deviceAuthorizationEndpoint("/oauth2/device_authorization")
				.deviceVerificationEndpoint("/oauth2/device_verification")
				.tokenEndpoint("/oauth2/token")
				.jwkSetEndpoint("/oauth2/jwks")
				.tokenRevocationEndpoint("/oauth2/revoke")
				.tokenIntrospectionEndpoint("/oauth2/introspect")
				.oidcClientRegistrationEndpoint("/connect/register")
				.oidcUserInfoEndpoint("/userinfo")
				.oidcLogoutEndpoint("/connect/logout");
	}
}
```

- OAuth2 authorizationEndpoint: `OAuth2AuthorizationEndpointConfigurer`
- OAuth2 deviceAuthorizationEndpoint
- OAuth2 deviceVerificationEndpoint
- OAuth2 tokenEndpoint
- OAuth2 tokenIntrospectionEndpoint
- OAuth2 tokenRevocationEndpoint
- OAuth2 authorization server metadata endpoint
- jwkSetEndpoint
- OpenID Connect 1.0 provider configuration endpoint
- oidcLogoutEndpoint
- oidcUserInfoEndpoint
- oidcClientRegistrationEndpoint

# Log In
* https://docs.spring.io/spring-security/reference/servlet/oauth2/login/index.html

# Resource Server
* code: `D:\workspace\rtfsc\spring-security\oauth2`

```groovy
implementation 'org.springframework.boot:spring-boot-starter-oauth2-resource-server'
```

# Client
* code: `D:\workspace\rtfsc\spring-security\oauth2`

```groovy
implementation 'org.springframework.boot:spring-boot-starter-oauth2-client'
```

# JWT

- [Nimbus JOSE + JWT](https://connect2id.com/products/nimbus-jose-jwt)
```xml
<groupId>com.nimbusds</groupId>
<artifactId>nimbus-jose-jwt</artifactId>
```